In [1]:
from __future__ import print_function
%matplotlib inline
import mdtraj as md
import numpy as np
import matplotlib.pyplot as plt
import scipy.cluster.hierarchy
from scipy.spatial.distance import squareform

In [2]:
traj = md.load('ntail.apo.protein.300K.h5')

/home/jwdell/anaconda3/envs/new_chem101/lib/python3.7/site-packages/mdtraj/core/trajectory.py:438: UserWarning: top= kwargs ignored since this file parser does not support it
  warnings.warn('top= kwargs ignored since this file parser does not support it')


In [3]:
# simple version for really small molecules
# mdtraj.org/1.9.4/examples/clustering.html

# distances = np.empty((traj.n_frames, traj.n_frames))
# for i in range(traj.n_frames):
#     distances[i] = md.rmsd(traj, traj, i)
# print('Max pairwise rmsd: %f nm' % np.max(distances))

In [4]:
# assert np.all(distances - distances.T < 1e-6)
# reduced_distances = squareform(distances, checks=False)

In [5]:
# linkage = scipy.cluster.hierarchy.linkage(reduced_distances, method='average')

In [6]:
# plt.title('RMSD Average linkage hierarchical clustering')
# _ = scipy.cluster.hierarchy.dendrogram(linkage, no_labels=True, count_sort='descendent')

In [7]:
# multi-pass version for beefier trajectories
# mdtraj.org/1.9.4/examples/two-pass-clustering.html
from __future__ import print_function
import random
from collections import defaultdict
import mdtraj as md
import numpy as np
import scipy.cluster.hierarchy

In [8]:
stride = 10
subsampled = md.load('ntail.apo.protein.300K.h5', stride=stride)

In [9]:
distances = np.empty((subsampled.n_frames, subsampled.n_frames))
for i in range(subsampled.n_frames):
    distances[i] = md.rmsd(subsampled, subsampled, i)

In [ ]:
n_clusters = 5
linkage = scipy.cluster.hierarchy.ward(distances)
labels = scipy.cluster.hierarchy.fcluster(linkage, t=n_clusters, criterion='maxclust')

In [ ]:
mapping = defaultdict(lambda : [])
for i, label in enumerate(labels):
    mapping[label].append(i)

In [ ]:
n_leaders_per_cluster = 2
leaders = md.Trajectory(xyz=np.empty((0, subsampled.n_atoms, 3)),
                        topology=subsampled.topology,unitcell_lengths=np.empty((0,3)),
                        unitcell_angles=np.empty((0,3)))

leader_labels = []
for label, indices in mapping.items():
    leaders = leaders.join(subsampled[np.random.choice(indices, n_leaders_per_cluster)])
    leader_labels.extend([label] * n_leaders_per_cluster)

print(leader_labels)

In [ ]:
# assign cluster identity to every frame
labels = []
for frame in md.iterload('ntail.apo.protein.300K.h5', chunk=1):
    labels.append(leader_labels[np.argmin(md.rmsd(leaders, frame, 0))])
labels = np.array(labels)

# print(labels)
# print(labels.shape)

In [ ]:
leaders.save('ntail.protein.leaders.pdb')

In [13]:
# switching to density-based approach
# scikit-learn.org/stable/modules/generated/sklearn.cluster.DBSCAN.html
from sklearn.cluster import DBSCAN

In [28]:
# eps chosen by trial and error...
# distances matrix from above used as X
db = DBSCAN(eps=.53,metric='precomputed').fit(distances)
labels = db.labels_

# Number of clusters in labels, ignoring noise if present.
n_clusters_ = len(set(labels)) - (1 if -1 in labels else 0)
n_noise_ = list(labels).count(-1)

print("Estimated number of clusters: %d" % n_clusters_)
print("Estimated number of noise points: %d" % n_noise_)

Estimated number of clusters: 5
Estimated number of noise points: 179
